This notebooks mimicks the behavior of ```sample_code/runHatsTrees.py``` to showcase few more nice features of rootpy, but the major downside of a somewhat slower performance with respect to a pure C++ implementation. Nonetheless this approach is very usefule when a limited amount of data is involved

We will use ROOT's TStopwatch to evaluate the performance of the code

In [3]:
from rootpy.io import root_open
from rootpy.tree import Tree
from rootpy.vector import LorentzVector
from ROOT import TStopwatch, TLorentzVector

In [4]:
tfile = root_open('root://cmseos.fnal.gov//store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig/sig_shuffled_0.root')
tree = tfile.tree
tree.SetBranchStatus("*",0)
for name in ["jetAK4_pt", 'jetAK4_eta', "jetAK4_mass", "jetAK4_phi",
             "jetAK4_e", "jetAK4_jec", "jetAK4_IDLoose"]:
    tree.SetBranchStatus(name, 1)

rootpy redefines the way TTrees are defined filled, allowing an easier and more pythonic syntax 

In [18]:
hatsDijetsFile = root_open('output.root', "RECREATE") # also 'w', like with normal files works
hatsDijetsTree = Tree("hatsDijets") #the title is assigned automatically 
hatsDijetsTree.create_branches({
    'leadingEta' : 'F',
    'subleadingEta' : 'F',
    'cosThetaStar' : 'F',
    'dijetEtBalance' : 'F',
    'dijetMass' : 'F',
})

clock = TStopwatch()
clock.Start()
leadingJet    = LorentzVector()
subleadingJet = LorentzVector()
evaluated = 0
for entry in tree:
    evaluated += 1
    leadingJet.SetPtEtaPhiE(
        entry.jetAK4_pt[0], 
        entry.jetAK4_eta[0], 
        entry.jetAK4_phi[0], 
        entry.jetAK4_e[0]
    )
    subleadingJet.SetPtEtaPhiE(
        entry.jetAK4_pt[1], 
        entry.jetAK4_eta[1], 
        entry.jetAK4_phi[1], 
        entry.jetAK4_e[1]
    )
      
    hatsDijetsTree.leadingEta = leadingJet.Eta()
    hatsDijetsTree.subleadingEta = subleadingJet.Eta()
    dijet = leadingJet + subleadingJet
    hatsDijetsTree.dijetMass = dijet.M()
    hatsDijetsTree.dijetEtBalance = subleadingJet.Et() / leadingJet.Et()
    boostedLeading = leadingJet
    boostedLeading.Boost(-(dijet.BoostVector()))
    hatsDijetsTree.cosThetaStar = boostedLeading.Pz()/boostedLeading.P()
    hatsDijetsTree.fill()
hatsDijetsFile.cd()
hatsDijetsTree.write()
hatsDijetsFile.close()
clock.Stop()

Now, let's see how our code performs

In [19]:
print 'Tree loop profiling stats:'
print '# entries examined:', evaluated
print 'Real Time used:', clock.RealTime(), '(per event: %.5f)' % (clock.RealTime()/evaluated)
print 'CPU Time used:', clock.CpuTime(), '(per event: %.5f)' % (clock.CpuTime()/evaluated)

Tree loop profiling stats:
# entries examined: 16921
Real Time used: 3.54225492477 (per event: 0.00021)
CPU Time used: 3.81 (per event: 0.00023)


You can see that we defined the lorentz vectors outside the for loop, this is done because LorentzVector creation is an intensive process in ROOT, rootpy makes it way faster! 
Try to re-run the two previous code blocks with the following changes:
   * move the LorentzVectors inside the for loop
   * replace rootpy's LorentzVectors with ROOT's TLorentzVector (inside and outside the for loop)
And see how the code performance changes